In [35]:
import numpy as np
import h5py
import pprint
import json

## From keras/saving/legacy/hdf5_format.py:
### load_weights_from_hdf5_group

In [23]:
f = h5py.File('simple_audio.h5')
list(f.attrs), list(f.keys())

(['backend', 'keras_version', 'model_config', 'training_config'],
 ['model_weights', 'optimizer_weights'])

In [37]:
f.attrs['backend'], f.attrs['keras_version']

('tensorflow', '2.12.0')

## Get the model config

In [36]:
model_config = json.loads(f.attrs['model_config'])
pprint.pp(model_config)

{'class_name': 'Sequential',
 'config': {'name': 'sequential',
            'layers': [{'class_name': 'InputLayer',
                        'config': {'batch_input_shape': [None, 124, 129, 1],
                                   'dtype': 'float32',
                                   'sparse': False,
                                   'ragged': False,
                                   'name': 'input_1'}},
                       {'class_name': 'Resizing',
                        'config': {'name': 'resizing',
                                   'trainable': True,
                                   'dtype': 'float32',
                                   'height': 32,
                                   'width': 32,
                                   'interpolation': 'bilinear',
                                   'crop_to_aspect_ratio': False}},
                       {'class_name': 'Normalization',
                        'config': {'name': 'normalization',
                                   

## Get the training config

In [38]:
training_config = json.loads(f.attrs['training_config'])
pprint.pp(training_config)

{'loss': {'class_name': 'SparseCategoricalCrossentropy',
          'config': {'reduction': 'auto',
                     'name': 'sparse_categorical_crossentropy',
                     'from_logits': True,
                     'ignore_class': None}},
 'metrics': [[{'class_name': 'MeanMetricWrapper',
               'config': {'name': 'accuracy',
                          'dtype': 'float32',
                          'fn': 'sparse_categorical_accuracy'}}]],
 'weighted_metrics': None,
 'loss_weights': None,
 'optimizer_config': {'class_name': 'Custom>Adam',
                      'config': {'name': 'Adam',
                                 'weight_decay': None,
                                 'clipnorm': None,
                                 'global_clipnorm': None,
                                 'clipvalue': None,
                                 'use_ema': False,
                                 'ema_momentum': 0.99,
                                 'ema_overwrite_frequency': None,
   

## Get the layer names

In [45]:
def load_attributes_from_hdf5_group(group, name):
    """Loads attributes of the specified name from the HDF5 group.

    This method deals with an inherent problem
    of HDF5 file which is not able to store
    data larger than HDF5_OBJECT_HEADER_LIMIT bytes.

    Args:
        group: A pointer to a HDF5 group.
        name: A name of the attributes to load.

    Returns:
        data: Attributes data.
    """
    if name in group.attrs:
        data = [
            n.decode("utf8") if hasattr(n, "decode") else n
            for n in group.attrs[name]
        ]
    else:
        data = []
        chunk_id = 0
        while "%s%d" % (name, chunk_id) in group.attrs:
            data.extend(
                [
                    n.decode("utf8") if hasattr(n, "decode") else n
                    for n in group.attrs["%s%d" % (name, chunk_id)]
                ]
            )
            chunk_id += 1
    return data

In [47]:
layer_names = load_attributes_from_hdf5_group(model_weights, 'layer_names')
layer_names

['resizing',
 'normalization',
 'conv2d',
 'conv2d_1',
 'max_pooling2d',
 'dropout',
 'flatten',
 'dense',
 'dropout_1',
 'dense_1']

## Get the weights

In [48]:
def load_subset_weights_from_hdf5_group(f):
    """Load layer weights of a model from hdf5.

    Args:
        f: A pointer to a HDF5 group.

    Returns:
        List of NumPy arrays of the weight values.

    Raises:
        ValueError: in case of mismatch between provided model
            and weights file.
    """
    weight_names = load_attributes_from_hdf5_group(f, "weight_names")
    return [np.asarray(f[weight_name]) for weight_name in weight_names]

In [85]:
for layer_name in layer_names:
    group = model_weights[layer_name]
    data = load_subset_weights_from_hdf5_group(group)
    print('%s:' % layer_name)
    print('data sets: %d' % len(data))
    for dset in data:
        print('======> %s' % str(dset.shape))
        print('%s' % dset)
    print('')

resizing:
data sets: 0

normalization:
data sets: 3
======> (1,)
[0.12540944]
======> (1,)
[0.58403146]
======> ()
102374400

conv2d:
data sets: 2
======> (3, 3, 1, 32)
[[[[ 2.32063651e-01  5.38834855e-02 -7.15002045e-02  9.31283087e-02
     1.45333344e-02  1.38282090e-01 -6.84894919e-02  1.20624304e-01
     1.57480627e-01  1.90342858e-01  6.29545078e-02 -3.18617709e-02
     5.90949357e-02  3.22743282e-02  1.27899140e-01 -4.63170335e-02
     1.19583070e-01 -1.90551370e-01  6.68633059e-02  1.18668735e-01
    -4.82094511e-02 -1.82905525e-01  1.88664749e-01 -1.28059238e-01
     2.14300379e-01  4.51913755e-03 -1.62543401e-01  3.14705878e-01
    -6.57505691e-02  1.10588342e-01 -2.48272449e-01 -1.05037615e-01]]

  [[ 6.25558048e-02 -1.91458195e-01  2.17498392e-01 -1.26596689e-01
    -1.18848331e-01  3.96420322e-02  1.55447841e-01  1.81929991e-01
     9.75919813e-02  1.80038214e-01  8.30860808e-02  9.25142989e-02
    -4.25422601e-02  2.20890567e-01 -9.45610926e-02  9.23718698e-03
     7.66803